In [1]:
import random as rd
import re
import math
import string
import pandas as pd

def k_means(tweets, k=4, max_iterations=50):

    centroids = []

    # initialization, assign random tweets as centroids
    count = 0
    hash_map = dict()
    while count < k:
        random_tweet_idx = rd.randint(0, len(tweets) - 1)
        if random_tweet_idx not in hash_map:
            count += 1
            hash_map[random_tweet_idx] = True
            centroids.append(tweets[random_tweet_idx])

    iter_count = 0
    prev_centroids = []

    # run the iterations until not converged or until the max iteration in not reached
    while (is_converged(prev_centroids, centroids)) == False and (iter_count < max_iterations):

        print("running iteration " + str(iter_count))

        # assignment, assign tweets to the closest centroids
        clusters = assign_cluster(tweets, centroids)

        # to check if k-means converges, keep track of prev_centroids
        prev_centroids = centroids

        # update, update centroid based on clusters formed
        centroids = update_centroids(clusters)
        iter_count = iter_count + 1

    if (iter_count == max_iterations):
        print("max iterations reached, K means not converged")
    else:
        print("converged")

    sse = compute_SSE(clusters)
    
    return clusters, sse


def is_converged(prev_centroid, new_centroids):

    # false if lengths are not equal
    if len(prev_centroid) != len(new_centroids):
        return False

    # iterate over each entry of clusters and check if they are same
    for c in range(len(new_centroids)):
        if " ".join(new_centroids[c]) != " ".join(prev_centroid[c]):
            return False

    return True


def assign_cluster(tweets, centroids):

    clusters = dict()

    # for every tweet iterate each centroid and assign closest centroid to a it
    for t in range(len(tweets)):
        min_dis = math.inf
        cluster_idx = -1;
        for c in range(len(centroids)):
            dis = getDistance(centroids[c], tweets[t])
            # look for a closest centroid for a tweet

            if centroids[c] == tweets[t]:
                # print("tweet and centroid are equal with c: " + str(c) + ", t" + str(t))
                cluster_idx = c
                min_dis = 0
                break

            if dis < min_dis:
                cluster_idx = c
                min_dis = dis

        # randomise the centroid assignment to a tweet if nothing is common
        if min_dis == 1:
            cluster_idx = rd.randint(0, len(centroids) - 1)

        # assign the closest centroid to a tweet
        clusters.setdefault(cluster_idx, []).append([tweets[t]])
        # print("tweet t: " + str(t) + " is assigned to cluster c: " + str(cluster_idx))
        # add the tweet distance from its closest centroid to compute sse in the end
        last_tweet_idx = len(clusters.setdefault(cluster_idx, [])) - 1
        clusters.setdefault(cluster_idx, [])[last_tweet_idx].append(min_dis)

    return clusters


def update_centroids(clusters):

    centroids = []

    # iterate each cluster and check for a tweet with closest distance sum with all other tweets in the same cluster
    # select that tweet as the centroid for the cluster
    for c in range(len(clusters)):
        min_dis_sum = math.inf
        centroid_idx = -1

        # to avoid redundant calculations
        min_dis_dp = []

        for t1 in range(len(clusters[c])):
            min_dis_dp.append([])
            dis_sum = 0
            # get distances sum for every of tweet t1 with every tweet t2 in a same cluster
            for t2 in range(len(clusters[c])):
                if t1 != t2:
                    if t2 < t1:
                        dis = min_dis_dp[t2][t1]
                    else:
                        dis = getDistance(clusters[c][t1][0], clusters[c][t2][0])

                    min_dis_dp[t1].append(dis)
                    dis_sum += dis
                else:
                    min_dis_dp[t1].append(0)

            # select the tweet with the minimum distances sum as the centroid for the cluster
            if dis_sum < min_dis_sum:
                min_dis_sum = dis_sum
                centroid_idx = t1

        # append the selected tweet to the centroid list
        centroids.append(clusters[c][centroid_idx][0])
    return centroids


def getDistance(tweet1, tweet2):

    # get the intersection
    intersection = set(tweet1).intersection(tweet2)

    # get the union
    union = set().union(tweet1, tweet2)

    # return the jaccard distance
    return 1 - (len(intersection) / len(union))


def compute_SSE(clusters):

    sse = 0
    # iterate every cluster 'c', compute SSE as the sum of square of distances of the tweet from it's centroid
    for c in range(len(clusters)):
        for t in range(len(clusters[c])):
            sse = sse + (clusters[c][t][1] * clusters[c][t][1])

    return sse


if __name__ == '__main__':

    tweets = pd.read_csv('arizona_clean.csv')
    tweets = tweets['tweet']

    # default number of experiments to be performed
    experiments = 1

    # default value of K for K-means
    k = 14

    # for every experiment 'e', run K-means
    for e in range(experiments):

        print("------ Running K means for experiment no. " + str((e + 1)) + " for k = " + str(k))

        clusters, sse = k_means(tweets, k)

        # for every cluster 'c', print size of each cluster
        for c in range(len(clusters)):
            print(str(c+1) + ": ", str(len(clusters[c])) + " tweets")
            # # to print tweets in a cluster
            # for t in range(len(clusters[c])):
            #     print("t" + str(t) + ", " + (" ".join(clusters[c][t][0])))

        print("--> SSE : " + str(sse))
        print('\n')

        # increment k after every experiment
        k = k + 1

------ Running K means for experiment no. 1 for k = 14
running iteration 0
running iteration 1
running iteration 2
converged
1:  448 tweets
2:  362 tweets
3:  88 tweets
4:  60 tweets
5:  52 tweets
6:  772 tweets
7:  44 tweets
8:  576 tweets
9:  259 tweets
10:  385 tweets
11:  302 tweets
12:  106 tweets
13:  221 tweets
14:  98 tweets
--> SSE : 71.07763817594517




In [2]:
clusters[0][1]

['new video soon makeupbrush ursula disneybound tutorialmakeup grwm corona de tucson arizona tcoycgklmsow',
 0.08333333333333337]

In [3]:
df = pd.DataFrame()
tweets_lsit = []
score_list = []
for i in range(len(clusters)):
    for j in range(len(clusters[i])):
        tweets_lsit.append(clusters[i][j][0])
        score_list.append(i)
        
df['tweets'] = tweets_lsit
df['cluster'] = score_list

In [4]:
def get_top_features_cluster(tf_idf_array, prediction, n_feats):
    labels = np.unique(prediction)
    dfs = []
    for label in labels:
        id_temp = np.where(prediction==label) # indices for each cluster
        x_means = np.mean(tf_idf_array[id_temp], axis = 0) # returns average score across cluster
        sorted_means = np.argsort(x_means)[::-1][:n_feats] # indices with top 20 scores
        features = tf_idf_vectorizor.get_feature_names()
        best_features = [(features[i], x_means[i]) for i in sorted_means]
        df = pd.DataFrame(best_features, columns = ['features', 'score'])
        dfs.append(df)
    return dfs
dfs = get_top_features_cluster(tf_idf_array, prediction, 15)

NameError: name 'tf_idf_array' is not defined

In [11]:
word_count=[]
for index,rows in df.iterrows():
    sent=rows.get("tweets")
    word_count.append(Counter(word_tokenize(sent)))
word_count

[Counter({'sinc': 1,
          'plan': 1,
          'weekend': 1,
          'cancel': 1,
          'due': 1,
          'covid': 1,
          'tri': 1,
          'step': 1,
          'yesterday': 1,
          'tcozxaolzl': 1}),
 Counter({'quarantin': 1,
          'day': 1,
          'nogal': 1,
          'arizona': 1,
          'tcoqiocjdtd': 1}),
 Counter({'paint': 1,
          'sip': 1,
          'deliveri': 1,
          'packag': 1,
          'worksnartebella': 1,
          'tucsonarizona': 1,
          'parkplac': 1,
          'tucsoncoronageddon': 1,
          'tcorgpeouajsl': 1}),
 Counter({'weur': 1,
          'evalu': 1,
          'condit': 1,
          'result': 1,
          'covid': 1,
          'coronavirus': 1,
          'tcojqlbudvq': 1}),
 Counter({'social': 1,
          'distanc': 1,
          'favsuufef': 1,
          'clifton': 1,
          'arizona': 1,
          'tcorstsltdb': 1}),
 Counter({'i': 1,
          'drink': 1,
          'better': 1,
          'wine': 1,
   

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\warri\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [5]:
import nltk